In [26]:
import numpy
import requests
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anany\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
job_postings = pd.read_csv('linkedin_job_postings.csv')
job_postings.head()


,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type
0,https://www.linkedin.com/jobs/view/account-exe...,2024-01-21 07:12:29.00256+00,t,t,f,Account Executive - Dispensing (NorCal/Norther...,BD,"San Diego, CA",2024-01-15,Coronado,United States,Color Maker,Mid senior,Onsite
1,https://www.linkedin.com/jobs/view/registered-...,2024-01-21 07:39:58.88137+00,t,t,f,Registered Nurse - RN Care Manager,Trinity Health MI,"Norton Shores, MI",2024-01-14,Grand Haven,United States,Director Nursing Service,Mid senior,Onsite
2,https://www.linkedin.com/jobs/view/restaurant-...,2024-01-21 07:40:00.251126+00,t,t,f,RESTAURANT SUPERVISOR - THE FORKLIFT,Wasatch Adaptive Sports,"Sandy, UT",2024-01-14,Tooele,United States,Stand-In,Mid senior,Onsite
3,https://www.linkedin.com/jobs/view/independent...,2024-01-21 07:40:00.308133+00,t,t,f,Independent Real Estate Agent,Howard Hanna | Rand Realty,"Englewood Cliffs, NJ",2024-01-16,Pinehurst,United States,Real-Estate Clerk,Mid senior,Onsite
4,https://www.linkedin.com/jobs/view/group-unit-...,2024-01-19 09:45:09.215838+00,f,f,f,Group/Unit Supervisor (Systems Support Manager...,"IRS, Office of Chief Counsel","Chamblee, GA",2024-01-17,Gadsden,United States,Supervisor Travel-Information Center,Mid senior,Onsite


In [28]:

job_skills = pd.read_csv('job_skills.csv')
job_skills.head()

,job_link,job_skills
0,https://www.linkedin.com/jobs/view/housekeeper...,"Building Custodial Services, Cleaning, Janitor..."
1,https://www.linkedin.com/jobs/view/assistant-g...,"Customer service, Restaurant management, Food ..."
2,https://www.linkedin.com/jobs/view/school-base...,"Applied Behavior Analysis (ABA), Data analysis..."
3,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Engineering, Project Controls, Sche..."
4,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Assembly, Point to point wiring, St..."


In [29]:
df_merged = pd.merge(job_skills, job_postings[['job_link', 'job_title', 'company', 'job_level']], on='job_link', how='inner')
df_merged.head(30)

,job_link,job_skills,job_title,company,job_level
0,https://www.linkedin.com/jobs/view/housekeeper...,"Building Custodial Services, Cleaning, Janitor...",Housekeeper I - PT,Jacksonville State University,Mid senior
1,https://www.linkedin.com/jobs/view/assistant-g...,"Customer service, Restaurant management, Food ...",Assistant General Manager - Huntington 4131,Ruby Tuesday,Mid senior
2,https://www.linkedin.com/jobs/view/school-base...,"Applied Behavior Analysis (ABA), Data analysis...",School-based Behavior Analyst,CCRES Educational and Behavioral Health Services,Mid senior
3,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Engineering, Project Controls, Sche...",Electrical Deputy Engineering Group Supervisor,Energy Jobline,Mid senior
4,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Assembly, Point to point wiring, St...",Electrical Assembly Lead,Sanmina,Mid senior
5,https://www.linkedin.com/jobs/view/senior-lead...,"Access Control, Video Management Systems, IPba...",Senior Lead Technician/Programmer,Security 101,Mid senior
6,https://www.linkedin.com/jobs/view/program-con...,"Consultation, Supervision, InService Training,...",Program Consultant,Methodist Family Health,Mid senior
7,https://www.linkedin.com/jobs/view/veterinary-...,"Veterinary Receptionist, AAHAaccredited, Custo...",Veterinary Receptionist,WellHaven Pet Health,Mid senior
8,https://www.linkedin.com/jobs/view/sr-technici...,"Optical Inspection Equipment Programming, MS E...","Sr. Technician, Receiving & Inspection",Abbott,Associate
9,https://www.linkedin.com/jobs/view/experienced...,"HVAC, troubleshooting, Preventative maintenanc...",Experienced HVAC Service Technician,Lane Valente Industries,Mid senior


In [30]:
df_merged_sample = df_merged.sample(frac=0.05, random_state=42)
# df_merged_sample.to_csv('merged_jobset_sample.csv')

In [31]:
df_merged_sample_sample = df_merged_sample.sample(n=1000, random_state=42)

In [32]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

In [33]:
ps = PorterStemmer()

In [37]:
def cleaning(txt):
    txt = txt.replace("/", " ")
    cleaned_txt = re.sub(r'[^a-zA-Z0-9\s]', '', txt)  # Removing all non-alphanumeric characters except spaces and parentheses
    tokens = nltk.word_tokenize(cleaned_txt.lower())  # Tokenizing the cleaned text and converting to lowercase
    steaming = [ps.stem(word) for word in tokens if word not in stopwords.words('english')]  # Applying stemming and removing stopwords
    return ' '.join(steaming)  # Returning the cleaned and processed text as a single string

In [39]:
cleaning('AdkjAKie am not typing in 3daA\ddi()')


'adkjaki type 3daaddi'

In [53]:
df_merged_sample_sample['cleaned_job_title'] = df_merged_sample_sample['job_title'].apply(lambda x: cleaning(x))

df_merged_sample_sample['cleaned_job_skills'] = df_merged_sample_sample['job_skills'].astype(str).apply(lambda x: cleaning(x))



df_merged_sample_sample.head()

,job_link,job_skills,job_title,company,job_level,cleaned_job_title,combined_title_skills,cleaned_job_skills
517804,https://www.linkedin.com/jobs/view/host-hostes...,"Hospitality, Customer service, Communication, ...",Host/Hostess - Seater,Ruth's Chris Steak House,Mid senior,host hostess seater,"host hostess seater Hospitality, Customer serv...",hospit custom servic commun teamwork restaur o...
706641,https://www.linkedin.com/jobs/view/corporate-c...,"Corporate Legal, State and Federal Relations, ...",Corporate Compliance and Litigation Manager,JDHuntr In-House Counsel Jobs,Mid senior,corpor complianc litig manag,"corpor complianc litig manag Corporate Legal, ...",corpor legal state feder relat complianc litig...
320994,https://www.linkedin.com/jobs/view/host-at-tex...,"Customer service, Teamwork, Communication, Att...",Host,Texas Roadhouse,Mid senior,host,"host Customer service, Teamwork, Communication...",custom servic teamwork commun attent detail ti...
599670,https://www.linkedin.com/jobs/view/quality-pro...,"Quality Program Manager, Food Safety Programs,...",Quality Program Manager,Jobot,Mid senior,qualiti program manag,"qualiti program manag Quality Program Manager,...",qualiti program manag food safeti program gfsi...
828511,https://www.linkedin.com/jobs/view/operations-...,"Retail experience, Management experience, Comm...",OPERATIONS ASSISTANT MANAGER,Dollar Tree Stores,Mid senior,oper assist manag,"oper assist manag Retail experience, Managemen...",retail experi manag experi commun skill interp...


In [54]:
# Importing the required modules from Scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
df_merged_sample_sample['combined_title_skills'] = df_merged_sample_sample['cleaned_job_title'] + ' ' + df_merged_sample_sample['cleaned_job_skills']
df_merged_sample_sample.head()

,job_link,job_skills,job_title,company,job_level,cleaned_job_title,combined_title_skills,cleaned_job_skills
517804,https://www.linkedin.com/jobs/view/host-hostes...,"Hospitality, Customer service, Communication, ...",Host/Hostess - Seater,Ruth's Chris Steak House,Mid senior,host hostess seater,host hostess seater hospit custom servic commu...,hospit custom servic commun teamwork restaur o...
706641,https://www.linkedin.com/jobs/view/corporate-c...,"Corporate Legal, State and Federal Relations, ...",Corporate Compliance and Litigation Manager,JDHuntr In-House Counsel Jobs,Mid senior,corpor complianc litig manag,corpor complianc litig manag corpor legal stat...,corpor legal state feder relat complianc litig...
320994,https://www.linkedin.com/jobs/view/host-at-tex...,"Customer service, Teamwork, Communication, Att...",Host,Texas Roadhouse,Mid senior,host,host custom servic teamwork commun attent deta...,custom servic teamwork commun attent detail ti...
599670,https://www.linkedin.com/jobs/view/quality-pro...,"Quality Program Manager, Food Safety Programs,...",Quality Program Manager,Jobot,Mid senior,qualiti program manag,qualiti program manag qualiti program manag fo...,qualiti program manag food safeti program gfsi...
828511,https://www.linkedin.com/jobs/view/operations-...,"Retail experience, Management experience, Comm...",OPERATIONS ASSISTANT MANAGER,Dollar Tree Stores,Mid senior,oper assist manag,oper assist manag retail experi manag experi c...,retail experi manag experi commun skill interp...


In [66]:
pd.set_option('display.max_colwidth', None)
print(df_merged_sample_sample.head(1)['cleaned_job_skills'])

517804    hospit custom servic commun teamwork restaur oper dine etiquett telephon etiquett attent detail problemsolv time manag flexibl stamina abil work fastpac environ
Name: cleaned_job_skills, dtype: object


In [57]:

# Creating an instance of the TfidfVectorizer class
tfidf = TfidfVectorizer()

# Creating a matrix of TF-IDF features for the 'new_col' column in the dataframe
matrix = tfidf.fit_transform(df_merged_sample_sample['combined_title_skills'])

# Computing the cosine similarity matrix for the TF-IDF features
similarity = cosine_similarity(matrix)

In [60]:
print(matrix)

  (0, 1755)	0.13349663693298908
  (0, 1887)	0.17456199541174025
  (0, 5220)	0.10155756147419233
  (0, 207)	0.1291882983550608
  (0, 4535)	0.1919440011986894
  (0, 1960)	0.13387664053785656
  (0, 2876)	0.06460581046487934
  (0, 4801)	0.11186456420687328
  (0, 3767)	0.1240605751816227
  (0, 1466)	0.12591021771342661
  (0, 541)	0.12987608435367604
  (0, 4738)	0.219272667021285
  (0, 1801)	0.6004843873715002
  (0, 1507)	0.26211523963141997
  (0, 3343)	0.11842900741346782
  (0, 4052)	0.18487448434728165
  (0, 4720)	0.11164025042625107
  (0, 1122)	0.06290720296472879
  (0, 4321)	0.08336139129849367
  (0, 1342)	0.09051918541857724
  (0, 2335)	0.1725801872668554
  (0, 4256)	0.3002421936857501
  (0, 2338)	0.271400266351133
  (0, 2337)	0.23327331229680284
  (1, 3730)	0.12067915918399962
  :	:
  (999, 2971)	0.2140175160449616
  (999, 4190)	0.18391787668080609
  (999, 2670)	0.19376059436941925
  (999, 1771)	0.12688675900792257
  (999, 3573)	0.119155323398735
  (999, 3604)	0.09515796488456346
  (99

,job_link,job_skills,job_title,company,job_level,cleaned_job_title,combined_title_skills
517804,https://www.linkedin.com/jobs/view/host-hostes...,"Hospitality, Customer service, Communication, ...",Host/Hostess - Seater,Ruth's Chris Steak House,Mid senior,host hostess seater,"host hostess seater Hospitality, Customer serv..."
706641,https://www.linkedin.com/jobs/view/corporate-c...,"Corporate Legal, State and Federal Relations, ...",Corporate Compliance and Litigation Manager,JDHuntr In-House Counsel Jobs,Mid senior,corpor complianc litig manag,"corpor complianc litig manag Corporate Legal, ..."
320994,https://www.linkedin.com/jobs/view/host-at-tex...,"Customer service, Teamwork, Communication, Att...",Host,Texas Roadhouse,Mid senior,host,"host Customer service, Teamwork, Communication..."
599670,https://www.linkedin.com/jobs/view/quality-pro...,"Quality Program Manager, Food Safety Programs,...",Quality Program Manager,Jobot,Mid senior,qualiti program manag,"qualiti program manag Quality Program Manager,..."
828511,https://www.linkedin.com/jobs/view/operations-...,"Retail experience, Management experience, Comm...",OPERATIONS ASSISTANT MANAGER,Dollar Tree Stores,Mid senior,oper assist manag,"oper assist manag Retail experience, Managemen..."


In [58]:
similarity

array([[1.        , 0.02662588, 0.20186074, ..., 0.0278744 , 0.01487489,
        0.18368636],
       [0.02662588, 1.        , 0.02387967, ..., 0.0216617 , 0.04007533,
        0.054004  ],
       [0.20186074, 0.02387967, 1.        , ..., 0.03169222, 0.02080036,
        0.15013975],
       ...,
       [0.0278744 , 0.0216617 , 0.03169222, ..., 1.        , 0.01562012,
        0.05533636],
       [0.01487489, 0.04007533, 0.02080036, ..., 0.01562012, 1.        ,
        0.03512407],
       [0.18368636, 0.054004  , 0.15013975, ..., 0.05533636, 0.03512407,
        1.        ]])

In [67]:
sample_skills_input = 'python java teamwork agile communications'


In [70]:

# Function to generate job recommendations
def recommendation(skills_input):
    skills_input = skills_input.apply(lambda x: cleaning(x))
    input_vector = tfidf.fit_transform(skills_input)
    
    similarity_scores = cosine_similarity(input_vector, matrix)
    similarity_scores = similarity_scores.tolist()
    distances = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)  # Calculate cosine similarity with other job titles
    
    jobs = []  # List to store the recommended job titles
    for i in distances:
        jobs.append(df_merged_sample_sample.iloc[i[0]]['job_title'])  # Get the titles of the similar jobs
        
    return jobs

In [72]:
# recommendation(sample_skills_input)